# In Class Notebook, Week 14

You can always paste the URL of this notebook (https://github.com/UIUC-iSchool-DataViz/is445_spring2022/blob/master/week14/inClass_week14.ipynb ) into the nbviewer interface for a plain-text rendering:

https://kokes.github.io/nbviewer.js/viewer.html

In [1]:
import numpy as np
import pandas as pd
import ipyvolume

In [2]:
x,y,z = np.random.random((3,100))

In [4]:
ipyvolume.quickscatter(x,y,z,size=1, marker='sphere')

In [6]:
from sys import path

In [7]:
path

['/Users/jnaiman/is445_spring2022/week14',
 '/Users/jnaiman/opt/anaconda3/envs/DataViz/lib/python37.zip',
 '/Users/jnaiman/opt/anaconda3/envs/DataViz/lib/python3.7',
 '/Users/jnaiman/opt/anaconda3/envs/DataViz/lib/python3.7/lib-dynload',
 '',
 '/Users/jnaiman/opt/anaconda3/envs/DataViz/lib/python3.7/site-packages',
 '/Users/jnaiman/opt/anaconda3/envs/DataViz/lib/python3.7/site-packages/IPython/extensions',
 '/Users/jnaiman/.ipython']

In [8]:
path.append('/Users/jnaiman/Downloads/')
path

['/Users/jnaiman/is445_spring2022/week14',
 '/Users/jnaiman/opt/anaconda3/envs/DataViz/lib/python37.zip',
 '/Users/jnaiman/opt/anaconda3/envs/DataViz/lib/python3.7',
 '/Users/jnaiman/opt/anaconda3/envs/DataViz/lib/python3.7/lib-dynload',
 '',
 '/Users/jnaiman/opt/anaconda3/envs/DataViz/lib/python3.7/site-packages',
 '/Users/jnaiman/opt/anaconda3/envs/DataViz/lib/python3.7/site-packages/IPython/extensions',
 '/Users/jnaiman/.ipython',
 '/Users/jnaiman/Downloads/']

In [9]:
from solverlibs import read_in_galaxy_data

In [10]:
galaxy_file ='/Users/jnaiman/Downloads/galaxyFiles/outarrsnap_001_fac1n3.txt'
t_h, r_h, v_h, e_h, N, part_type = read_in_galaxy_data(galaxy_file)
# t_h = time
# r_h = x,y,z positional vector for each timestep and each particle
# v_h = vx,vy,vz for each particle, each timestep
# e_h = energy (physics element)
# N = number of particles
# part_type = type of particle

In [11]:
r_h.shape

(82, 3, 500)

In [12]:
N

82

In [14]:
np.unique(part_type)

array([1., 2.])

In [17]:
tstep = 50

# r_h [particles, x/y/z, time step out of 500]
x,y,z = r_h[:,0,tstep], r_h[:,1,tstep], r_h[:,2,tstep]

In [18]:
ipyvolume.quickscatter(x,y,z,size=1,marker='sphere')

In [19]:
ipyvolume.quiver?

In [22]:
ipyvolume.figure()
for i in range(r_h.shape[0]): # loop over all particles
    ipyvolume.quiver(r_h[i,0,:250], # ith particle, x position, up to 100 time steps
                    r_h[i,1,:250], # ith particle, y position, up to 100 time steps
                    r_h[i,2,:250], # ith particle, z position, up to 100 time steps
                    v_h[i,0,:250], # ith particle, velocity in the x-direction, up to 100 timesteps
                    v_h[i,1,:250], # ith particle, vy, up to 100
                    v_h[i,2,:250], size=1) # ith particle, vz, up to 100
ipyvolume.show()

In [23]:
v_h.shape

(82, 3, 500)

In [25]:
step = 50 # plot every "step"th timestep
Nt = v_h.shape[2] # 500, or full, timesteps

ipyvolume.figure()
for i in range(r_h.shape[0]): # loop over all particles
    ipyvolume.quiver(r_h[i,0,0:Nt:step], # ith particle, x position, 0 to Nt (500 timesteps) in steps of "step"
                    r_h[i,1,0:Nt:step], # ith particle, y position, 0 to Nt (500 timesteps) in steps of "step"
                    r_h[i,2,0:Nt:step], # ith particle, z position, 0 to Nt (500 timesteps) in steps of "step"
                    v_h[i,0,0:Nt:step], # ith particle, velocity in the x-direction, 0 to Nt (500 timesteps) in steps of "step"
                    v_h[i,1,0:Nt:step], # ith particle, vy, 0 to Nt (500 timesteps) in steps of "step"
                    v_h[i,2,0:Nt:step], size=1) # ith particle, vz, 0 to Nt (500 timesteps) in steps of "step"
ipyvolume.show()

## 3d animations with ipyvolume

In [27]:
# all particles, x position, all timesteps
r_h[:,0,:].shape

(82, 500)

In [28]:
r_h[:,0,:].T.shape # 'swaps' the number of particles and timesteps (Transpose of a matrix)

(500, 82)

In [36]:
step = 10 # only plot with every 10th time step

r = r_h[:,:,0:Nt:step] # all particles, x/y/z, subsampled 
v = v_h[:,:,0:Nt:step]

# r = r_h[:,:,Nt:0:-step] # all particles, x/y/z, subsampled 
# v = v_h[:,:,Nt:0:-step]

In [37]:
r_h.shape, r.shape, r[:,0,:].T.shape

((82, 3, 500), (82, 3, 50), (50, 82))

In [38]:
ipyvolume.figure()

s = ipyvolume.scatter(r[:,0,:].T, r[:,1,:].T, r[:,2,:].T,
                     marker='sphere')

ani = ipyvolume.animation_control(s,interval=200) # animate our scatter plot with 200 milisec invervals

ipyvolume.show()

## Adding widgets

In [43]:
ipyvolume.figure()
s = ipyvolume.scatter(r[:,0,25], r[:,1,25], r[:,2,25],marker='sphere', color='blue')
ipyvolume.show()

In [40]:
import ipywidgets

In [41]:
color = ipywidgets.ColorPicker()

In [42]:
color

ColorPicker(value='black')

In [44]:
s.color

array('blue', dtype='<U4')

In [45]:
color.value

'#2b2fa1'

In [46]:
ipywidgets.jslink((s,'color'),(color,'value'))

Link(source=(Scatter(color=array('blue', dtype='<U4'), color_selected=array('white', dtype='<U5'), geo='sphere…

In [ ]:
ipyvolume.figure()
s = ipyvolume.scatter(r[:,0,25], r[:,1,25], r[:,2,25],marker='sphere', color='blue')